In [1]:
import numpy as np
import pandas as pd

In [14]:
# Constantes: número máximo de iterações, erro mínimo e taxa de aprendizado
MAX_ITER = 10000
MIN_ERROR = 0.01
LEARNING_RATE = 0.1

In [40]:
np.random.seed(666)
# Função que lê a matriz de entrada de dados, decide número de neurônios de entrada, saída e camada oculta.
def read_data(csv_file_path):
    data = pd.read_csv(csv_file_path)
    data = data.values
    # Número de entradas, ignora a última coluna (que apresenta as classes)
    n_inputs = data.shape[1] - 1
    # Número de saídas, depende do número de classes únicas na última coluna
    n_outputs = len(np.unique(data[:, -1]))
    # Número de neurônios na camada oculta é a média geométrica entre o número de entradas e saídas
    # Valor é arredondado
    n_hidden = int(np.round(np.sqrt(n_inputs * n_outputs)))
    return data, (n_inputs, n_hidden, n_outputs)

# Função que inicializa os pesos da camada de forma aleatória
# Parâmetros utilizados são o número de neurônios da camada anterior e da camada atual
def initialize_weights(n_inputs, n_outputs):
    # Pesos são inicializados de forma ALEATÓRIA
    # Nenhum valor desses gerados aleatoriamente pode ser 0
    # Por isso é adicionada uma constante 0.01
    #weight = np.random.rand(n_outputs, n_inputs) + 0.01
    #weight = np.random.uniform(-0.01, 0.01, (n_outputs, n_inputs)) + 0.001
    weight = np.random.normal(scale=0.1, size=(n_outputs, n_inputs))
    #print('pesos', weight)
    return weight

# Função que calcula o valor 'net' de cada neurônio de uma determinada camada
# Parâmetros utilizados são os pesos da camada e os valores de entrada
def calculate_neurons_net(weights, inputs):
    # Multiplica-se os pesos pela entrada
    # O resultado é a soma ponderada dos valores de entrada
    # O resultado é a entrada para a função de ativação
    net = np.dot(weights, inputs)
    return net

# Função que aplica a função de ativação aos valores 'net' de cada neurônio de uma camada
# Parâmetros são os valores 'net' e a funlção de ativação utilizada
def apply_transfer_function(net_values, transfer_function):
    # Função de ativação é aplicada a cada valor 'net'
    # O resultado é o valor de saída de cada neurônio
    outputs = transfer_function(net_values)
    return outputs

# Os valores calculados na função anterior se tornam os inputs utilizados para o cálculo na última camada

# Função que calcula o erro de saída de cada neurônio
# Parâmetros são os valores de saída e os valores esperados
# É necessário utilizar a derivada da função de ativação para o cálculo do erro
def calculate_output_error(outputs, expected_outputs, transfer_function_derivative):
    # O erro é calculado subtraindo-se os valores de saída dos valores esperados
    # O resultado é multiplicado pela derivada da função de ativação
    # O resultado é o erro de saída de cada neurônio
    error = (expected_outputs - outputs) * transfer_function_derivative(outputs)
    return error

# Função que propaga o erro para a camada anterior
def propagate_error(weights, error, transfer_function_derivative):
    # O erro é multiplicado pelos pesos
    # O resultado é multiplicado pela derivada da função de ativação
    # O resultado é o erro propagado para a camada anterior
    error = np.dot(weights.T, error) * transfer_function_derivative(error)
    return error

# A função anterior é aplicada na ordem inversa da rede, começando pela última camada e depois
# sendo aplicada na camada oculta.

# Função que ajusta os pesos com base no erro calculado, taxa de aprendizado e valores da função de ativação
# Precisa dos valores de saída da camada anterior
def adjust_weights(weights, error, last_layer_output):
    # O erro é multiplicado pela taxa de aprendizado
    # O resultado é multiplicado pelo valor de saída de cada neurônio
    # O resultado é o ajuste dos pesos
    adjustment = LEARNING_RATE * np.vstack(error) * last_layer_output
    # Os pesos são ajustados
    weights += adjustment
    return weights

# Função para normalizaros dados por coluna
def normalize_data(data):
    # For each column, normalize the numbers to be between 0 and 1
    for i in range(data.shape[1] - 1):
        # Get the min and max values for the column
        min_val = np.min(data[:, i])
        max_val = np.max(data[:, i])
        # Subtract the min from each value in the column
        data[:, i] -= min_val
        # Divide each value in the column by the max
        data[:, i] /= (max_val - min_val)
    return data

# Essa função também é aplicada na ordem inversa da rede

Exemplo de aplicação das funções elaboradas

In [41]:
# Função de ativação tangente hiperbólica
def tanh(x):
    return np.tanh(x)

# Função de derivada da função de ativação tangente hiperbólica
def tanh_derivative(x):
    return (1 / np.cosh(x)) ** 2

# Função de ativação logística
def logistic(x):
    return 1 / (1 + np.exp(-x))

# Função de derivada da função de ativação logística
def logistic_derivative(x):
    return logistic(x) * (1 - logistic(x))

# Carregando o arquivo "treinamento.csv"
data, number_of_neurons = read_data('treinamento.csv')

# Normalize data to values between 0 and 1
data = normalize_data(data.astype(float))

# Pesos da camada oculta
hidden_layer_weights = initialize_weights(number_of_neurons[0], number_of_neurons[1])
# Pesos da camada de saída
output_layer_weights = initialize_weights(number_of_neurons[1], number_of_neurons[2])

# Manipulando data para que as classes sejam apresentadas de forma alternada
# Separando valores por classe
data_by_class = {}
for row in data:
    if row[-1] not in data_by_class:
        data_by_class[row[-1]] = np.array([row])
    data_by_class[row[-1]] = np.append(data_by_class[row[-1]], [row], axis=0)
# Classes
classes = list(data_by_class.keys())
# Número de classes
n_classes = len(classes)
# Começa com a primeira classe, depois a segunda, até chegar a última e depois retorna para a primeira
# Isso é feito para que a rede aprenda a classificar as classes de forma alternada
for itr in range(MAX_ITER):
    class_index = itr % n_classes
    # Classe atual
    current_class = classes[class_index]
    # Qual linha da classe atual será escolhida
    row_index = itr // n_classes
    # Linha atual, com exceção da última coluna, apresenta os inputs
    try:
        input_from_data = data_by_class[current_class][row_index][:-1]
    except IndexError:
        # Caso não existir, uma das classes foi esgotada. Para evitar
        # desbalanceamento, o treinamento é finalizado.
        break

    # Calculo de 'net' para a camada oculta
    hidden_layer_net = calculate_neurons_net(hidden_layer_weights, input_from_data)
    # Aplicação da função de ativação f(x) = x
    hidden_layer_outputs = apply_transfer_function(hidden_layer_net, tanh)
    
    # Calculo de 'net' para a camada de saída
    output_layer_net = calculate_neurons_net(output_layer_weights, hidden_layer_outputs)
    # Aplicação da função de ativação f(x) = x
    output_layer_outputs = apply_transfer_function(output_layer_net, tanh)

    # Cálculo do erro de saída
    # Os valores esperados são -1 para todas as classes, exceto a atual que será 1
    expected_outputs = np.array([-1] * n_classes)
    expected_outputs[class_index] = 1
    output_layer_error = calculate_output_error(output_layer_outputs, expected_outputs, tanh_derivative)
    # Cálculo do erro da camada oculta
    hidden_layer_error = propagate_error(output_layer_weights, output_layer_error, tanh_derivative)

    # Ajuste de pesos da camada de saída
    output_layer_weights = adjust_weights(output_layer_weights, output_layer_error, hidden_layer_outputs)
    # Ajuste de pesos da camada oculta
    hidden_layer_weights = adjust_weights(hidden_layer_weights, hidden_layer_error, input_from_data)

    # Erro total é dado pela soma dos quadrados dos erros de saída divido por 2
    total_error = np.sum(output_layer_error ** 2) / 2
    #print(total_error)
    
    # Se o erro total for menor que o erro mínimo, a rede é considerada treinada
    if total_error < MIN_ERROR:
        print("Chegou ao fim pois convergiu dentro do erro mínimo: ", total_error)
        break

if total_error >= MIN_ERROR:
    print("Chegou ao fim pois atingiu o número máximo de iterações: ", MAX_ITER)
    print("Erro total final: ", total_error)


Chegou ao fim pois convergiu dentro do erro mínimo:  0.00885807130802876


Testando a rede

In [49]:
# Testando a rede
# Carregando o arquivo "teste.csv"
data, number_of_neurons = read_data('teste.csv')
# Normalize data (all columns except last one) to values between 0 and 1
data = normalize_data(data.astype(float))

# Número de classes
n_classes = len(classes)
# Saída
output = []
# Número de erros
n_errors = 0
# Número de linhas
n_rows = len(data)
# Para cada linha do arquivo
for row in data:
    # Input da linha
    input_from_data = row[:-1]
    # Classe esperada
    expected_class = row[-1]
    # Calculo de 'net' para a camada oculta
    hidden_layer_net = calculate_neurons_net(hidden_layer_weights, input_from_data)
    # Aplicação da função de ativação f(x) = x
    hidden_layer_outputs = apply_transfer_function(hidden_layer_net, tanh)
    
    # Calculo de 'net' para a camada de saída
    output_layer_net = calculate_neurons_net(output_layer_weights, hidden_layer_outputs)
    # Aplicação da função de ativação f(x) = x
    output_layer_outputs = apply_transfer_function(output_layer_net, tanh)

    #np.argsort(np.max(x, axis=0))[-2]
    # Classe obtida
    # make so that the biggest value in output_layer_outputs is 1 and the others are -1
    output_layer_outputs = np.where(output_layer_outputs == np.max(output_layer_outputs), 1, -1)

    obtained_class = classes[np.argmax(output_layer_outputs)]
    # if expected_class is 1.0, use the second highest value~
    #if expected_class == 1.0:
    #    obtained_class = classes[np.argsort(output_layer_outputs)[-2]]
    # Se a classe obtida for igual a esperada, é um acerto
    output.append([obtained_class, expected_class])
    if obtained_class != expected_class:
        n_errors += 1

# Imprimindo a saída
print("Saída:")
print(output)
# Imprimindo o erro
print("Erro: ", n_errors / n_rows)

# Print outputs where the expected class is different from the obtained class
print("Saídas onde a classe esperada é diferente da obtida:")
for row in output:
    if row[0] != row[1]:
        print(row)

Saída:
[[3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [3.0, 1.0], [2.0, 2.0], [2.0, 2.0], [2.0, 2.0], [2.0, 2.0], [2.0, 2.0], [2.0, 2.0], [2.0, 2.0], [2.0, 2.0], [2.0, 2.0], [2.0, 2.0], [2.0, 2.0], [2.0, 2.0], [2.0, 2.0], [2.0, 2.0], [2.0, 2.0], [2.0, 2.0], [2.0, 2.0], [2.0, 2.0], [2.0, 2.0], [2.0, 2.0], [2.0, 2.0], [2.0, 2.0], [2.0, 2.0], [2.0, 2.0], [2.0, 2.0], [2.0, 2.0], [2.0, 2.

Exemplo apresentado na aula 17

In [22]:
# Dois neurônios na primeira camada
# Quatro neurônios na segunda camada
# Três neurônios na terceira camada
# Pesos da camada oculta
# Taxa de aprendizado igual a 1
LEARNING_RATE = 1

hidden_layer_weights = np.array([
    [1.1, -1.4],
    [3.6, -4.1],
    [2.1, 2.5],
    [0.9, -1.0]])
# Pesos da camada de saída
output_layer_weights = np.array([
    [1.2, 1.6, 4.3, 3.2]
])
# Entrada é 0, 1
input_from_data = np.array([0, 1])
# Calculo de 'net' para a camada oculta
hidden_layer_net = calculate_neurons_net(hidden_layer_weights, input_from_data)
print("Net da camada oculta: ", hidden_layer_net)
# Aplicação da função de ativação f(x) = x
hidden_layer_outputs = apply_transfer_function(hidden_layer_net, lambda x: x)
print("Saída da camada oculta: ", hidden_layer_outputs)
# Calculo de 'net' para a camada de saída
output_layer_net = calculate_neurons_net(output_layer_weights, hidden_layer_outputs)
print("Net da camada de saída: ", output_layer_net)
# Aplicação da função de ativação f(x) = x
output_layer_outputs = apply_transfer_function(output_layer_net, lambda x: x)
print("Saída da camada de saída: ", output_layer_outputs)

# Cálculo do erro de saída
output_layer_error = calculate_output_error(output_layer_outputs, np.array([1]), lambda x: 1)
print("Erro de saída: ", output_layer_error)
# Cálculo do erro da camada oculta
hidden_layer_error = propagate_error(output_layer_weights, output_layer_error, lambda x: 1)
print("Erro da camada oculta: ", hidden_layer_error)

# Ajuste de pesos da camada de saída
output_layer_weights = adjust_weights(output_layer_weights, output_layer_error, hidden_layer_outputs)
print("Novos pesos da camada de saída: ", output_layer_weights)
# Ajuste de pesos da camada oculta
hidden_layer_weights = adjust_weights(hidden_layer_weights, hidden_layer_error, input_from_data)
print("Novos pesos da camada oculta: ", hidden_layer_weights)


Net da camada oculta:  [-1.4 -4.1  2.5 -1. ]
Saída da camada oculta:  [-1.4 -4.1  2.5 -1. ]
Net da camada de saída:  [-0.69]
Saída da camada de saída:  [-0.69]
Erro de saída:  [1.69]
Erro da camada oculta:  [2.028 2.704 7.267 5.408]
Novos pesos da camada de saída:  [[-1.166 -5.329  8.525  1.51 ]]
Novos pesos da camada oculta:  [[ 1.1    0.628]
 [ 3.6   -1.396]
 [ 2.1    9.767]
 [ 0.9    4.408]]


Resultados bem próximos.